In [1]:
!nvidia-smi
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import torch
import numpy as np
import time
import os
import csv
import cv2
import copy
import argparse
import matplotlib.pyplot as plt
import random
import torch.optim as optim
import sys
from tqdm import tqdm
from visualize_single_image import*


######### !!!!!!!!!!!!!!!!!!!!!!!!!!!!! #########################
sys.path.append('/workspace/flat_feet/Unet/')
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# HISTORY_PATH = "/workspace/flat_feet/Unet/weight_manet/" # path_root of Unet weight
# HISTORY_PATH = "/workspace/flat_feet/Unet/weight_unet384/" # path_root of Unet weight
# HISTORY_PATH = "/workspace/flat_feet/Unet/weight_unet512/" # path_root of Unet weight
# HISTORY_PATH = "/workspace/flat_feet/Unet/weight_unet768/" # path_root of Unet weight
# HISTORY_PATH = "/workspace/flat_feet/Unet/weight_unet1024/" # path_root of Unet weight
# HISTORY_PATH = "/workspace/flat_feet/Unet/weight_unet384_finetune/" # path_root of Unet weight
# HISTORY_PATH = "/workspace/flat_feet/Unet/weight_unet512_finetune/" # path_root of Unet weight
# HISTORY_PATH = "/workspace/flat_feet/Unet/weight_unet1024_finetune/" # path_root of Unet weight
HISTORY_PATH = "/workspace/flat_feet/Unet/weight_unet1024_noSCSE/" # path_root of Unet weight
######### !!!!!!!!!!!!!!!!!!!!!!!!!!!!! #########################

from trainer import *
from loss import *
from Unet import *
from preprocessing import *
from datagenerater import *
from utils import *
from progressbar import Bar
from PIL import Image
# import SimpleITK as sitk

# HISTORY_PATH = "/mnt/nas125/InHwanKim/weight/rsm/segmentation(all)/"
# DIR = '/mnt/nas125/InHwanKim/data/rsm/'
# class_path = DIR + '/classes.csv'
# HISTORY_PATH = "/workspace/flat_feet/Unet/weight_origin/"


#테스트_데이터 로드
image_paths = []
gt_paths = []

max_size = 4000
if '384' in HISTORY_PATH:
    shift = patch_size = 384
elif '512' in HISTORY_PATH:
    shift = patch_size = 512
elif '1024' in HISTORY_PATH:
    shift = patch_size = 1024
else:
    shift = patch_size = 512
print(patch_size)

weight_paths = []
for i in range(25):
    path = HISTORY_PATH + str(i) + "/"
    model_name = os.listdir(path)[-1]
    modle_path = path + model_name
    weight_paths.append(modle_path)
    
for i in range(len(weight_paths)):
    print(weight_paths[i])

Wed Jan  5 04:03:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:02:00.0 Off |                  N/A |
| 43%   51C    P0    68W / 280W |      0MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:03:00.0 Off |                  N/A |
| 43%   

# 1. Inference Retinanet

In [ ]:
# !pip install natsort --user
# test Retinanet 1 stage
import glob
from tqdm import trange
from natsort import natsorted
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

######### !!!!!!!!!!!!!!!!!!!!!!!!!!!!! #########################
DIR = '/workspace/flat_feet/dataset/png_test/'
# DIR = '/workspace/flat_feet/dataset/Segmentation_210712/ExternalValidation/LERA_WBLR/refined/'
# DIR = '/workspace/flat_feet/dataset/Segmentation_210712/ExternalValidation/AMC_100_WBLR/refined/'

# ###########################for ROI tuning (from validset)###############################
# label_paths = natsorted(glob.glob(DIR+'label/*.npy'))[800:1023]
# # label_paths = natsorted(glob.glob(DIR+'label/*.npy'))[800:1023]
# image_paths = list()
# for idx in range(len(label_paths)):
#     image_paths.append(glob.glob(DIR+'image/'+label_paths[idx].split('/')[-1][:-4])[0])
# ########################################################################################

image_paths = natsorted(glob.glob(DIR + '*png'))
label_paths = natsorted(glob.glob("/workspace/flat_feet/dataset/flat_feet/label_test/*.npy"))
# image_paths = natsorted(glob.glob(DIR + 'image_test_2nd/*png'))
# label_paths = natsorted(glob.glob(DIR + "label_test_2nd/*npy"))
# image_paths = image_paths[:len(label_paths)]
print(len(image_paths),len(label_paths))
######### !!!!!!!!!!!!!!!!!!!!!!!!!!!!! #########################

model_path = "/workspace/flat_feet/pytorch-retinanet/weight/retinanet_120.pt"
model = load_model(model_path)

GT_Points = []
Predict_Points = []
device = torch.device('cuda')
model.to(device)

for i in tqdm(range(len(image_paths))):
    patient_distance = []
    Predict_Point = []
    
    Predict_Info = detect_image(model, image_paths[i])
    image_orig = cv2.imread(image_paths[i])
    
#     for j in range(Predict_Info.shape[0]-1):
    for j in range(25):
        P_X_point = int(((Predict_Info[j][1])+int(Predict_Info[j][3]))/2)
        P_Y_point = int(((Predict_Info[j][2])+int(Predict_Info[j][4]))/2)
        pr_point = [P_X_point,P_Y_point]
        Predict_Point.append(pr_point)
    
    Predict_Points.append(Predict_Point)
    try:
        label = np.load(label_paths[i])
        GT_Points.append(label)
    except:
        pass
    
#     # plots
#     fig, ax = plt.subplots(figsize=(16,10))
#     ax.imshow(image_orig)
#     j = 0
#     for j in range(25):
#         rect = patches.Rectangle((Predict_Info[j][1], Predict_Info[j][2]), Predict_Info[j][3]-Predict_Info[j][1], Predict_Info[j][4]-Predict_Info[j][2], linewidth=1, edgecolor='r', facecolor='none',alpha=1)
#         ax.add_patch(rect)

#     plt.axis('off')
#     plt.show()
    
print(np.array(Predict_Points).shape,np.array(GT_Points).shape)

In [ ]:
Predict_Info[j][3]-Predict_Info[j][1]

In [ ]:
Predict_Info[j][4]-Predict_Info[j][2]

In [ ]:
Predict_Info[j][1], Predict_Info[j][2]

In [ ]:
# Create figure and axes
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

fig, ax = plt.subplots(figsize=(16,10))

# Display the image
ax.imshow(image_orig)

# Create a Rectangle patch
Predict_Info
j = 0
for j in range(25):
    rect = patches.Rectangle((Predict_Info[j][1], Predict_Info[j][2]), Predict_Info[j][3]-Predict_Info[j][1], Predict_Info[j][4]-Predict_Info[j][2], linewidth=1, edgecolor='r', facecolor='none',alpha=1)
    # rect = patches.Rectangle((Predict_Info[j][1], Predict_Info[j][2]), 1024, 1024, linewidth=1, edgecolor='r', facecolor='none',alpha=1)
    ax.add_patch(rect)

plt.axis('off')
plt.show()

In [ ]:
# # view (you can skip)
# for i in range(len(image_paths)):
#     print(image_paths[i].split("/")[-1])
#     img = cv2.imread(image_paths[i])
#     for j in range(25):
#         cv2.circle(img,((Predict_Points[i][j][0]),(Predict_Points[i][j][1])),8,(255,0,0),-1 )
# #         cv2.circle(img,((GT_Points[i][j][0]),(GT_Points[i][j][1])),8,(0,0,255),-1 )
        
#     plt.figure(figsize=(14, 14))
#     plt.title('GT:Red, Pred:Blue,')
#     plt.axis('off')
#     plt.imshow(img)
#     plt.show()

# 2. Inference Unet(25 times)

In [ ]:
# 4010 , 1~22
# !pip install timm==0.3.0 --user

In [ ]:
# Predict_Point Retina
# ppoints Unet

In [ ]:
# class Resize(object):
#     def __call__(self, sample, x_size= 2048):
#         image = sample['image']
#         landmarks = sample['landmarks']
#         x_ori, y_ori,_ = image.shape
#         image = cv2.resize(image, (x_size,int((x_size*y_ori)/x_ori)))
#         landmarks = cv2.resize(landmarks, (x_size,int((x_size*y_ori)/x_ori)))
        
#         image_ = np.zeros((x_size,y_size))
#         landmarks_ = np.zeros((x_size,y_size))
#         print(image.shape,image_.shape)
        
#         if image.shape[1]>=y_size:
#             image_= image[-y_size:]
#             landmarks_= landmarks[-y_size:]
#         elif image.shape[1]<y_size:
#             image_[-y_size:]= image
#             landmarks_[-y_size:]= landmarks
            
#         image = np.expand_dims(image_,2)
#         landmarks = np.expand_dims(landmarks_,2)
        
#         plt.imshow(image)
#         plt.imshow(landmarks,alpha=0.3)
#         plt.show()
        
#         return {'image': image, 'landmarks': landmarks}

def Resize(image, x_size= 2048):
    y_ori, x_ori = image.shape
    image = cv2.resize(image, (x_size,int((x_size*y_ori)/x_ori)))
    ratio = x_size/x_ori
    return image, ratio

In [ ]:
# !pip install git+https://github.com/qubvel/segmentation_models.pytorch --user --quiet
# !pip install ipywidgets --user
#########################!!!!!!!!!!!!!!!!!!!!!!#################################
# model = UNet(n_channels = 1, n_classes = 1)
import segmentation_models_pytorch as smp
# model = smp.Unet(encoder_name='timm-tf_efficientnet_lite4',decoder_attention_type='scse', in_channels = 1, classes = 1)
model = smp.Unet(encoder_name='timm-tf_efficientnet_lite4',decoder_attention_type=None, in_channels = 1, classes = 1)
#########################!!!!!!!!!!!!!!!!!!!!!!#################################

ppoints = np.zeros((len(image_paths),25,2),dtype=np.uint32)

for LM in range(0,25):
    device = torch.device('cuda')
    model.load_state_dict(torch.load(weight_paths[LM]))
    model.to(device)

    for j in tqdm(range(len(image_paths))):
        temp_image = cv2.imread(image_paths[j],0)
        temp_image, ratio = Resize(temp_image)
        
        padding_image = np.zeros((max_size,max_size),dtype=np.uint8)
        padding_image[shift:temp_image.shape[0]+shift,shift:temp_image.shape[1]+shift] = temp_image 
                
        left_up = (int((Predict_Points[j][LM][0]*ratio-(patch_size/2) + shift)),int((Predict_Points[j][LM][1]*ratio-(patch_size/2)+ shift)))
        right_down = (int((Predict_Points[j][LM][0]*ratio+(patch_size/2)+ shift)),int((Predict_Points[j][LM][1]*ratio+(patch_size/2)+ shift)))

        image = padding_image[left_up[1]:right_down[1],left_up[0]:right_down[0]]
        if image.shape != (patch_size,patch_size):
            image = image[:patch_size,:patch_size]
        image = Pre_Processing(image)
        
        output = model(image)
        output= torch.sigmoid(output)
        pr_mask = output.squeeze().cpu().detach().numpy()[:,:].round()
                
        pr_x_mean,pr_y_mean = Get_Center_Points(pr_mask)
        
#         ppoints[j][LM][0] = left_up[0] + pr_x_mean - shift
#         ppoints[j][LM][1] = left_up[1] + pr_y_mean - shift
        ppoints[j][LM][0] = (left_up[0] + pr_x_mean - shift)/ratio 
        ppoints[j][LM][1] = (left_up[1] + pr_y_mean - shift)/ratio 

In [ ]:
# save and view (you can skip)
# max_idx = 22
max_idx = 25
radius = 16
alpha = 0.7

save_path = DIR + f"result_{max_idx}/gt_pred_patchsize{patch_size}/"
if not os.path.exists(save_path):
    os.makedirs(save_path)

for i in range(len(image_paths)):
    print(image_paths[i])
    img = cv2.imread(image_paths[i])
    lm_unet = lm_retina = lm_gt = np.zeros_like(img)
    out = img.copy()
    for j in range(max_idx):
        try:
            cv2.circle(img,((ppoints[i][j][0]),(ppoints[i][j][1])),radius, (255,0,0),-1 ) # Unet
            # cv2.circle(lm_unet,((ppoints[i][j][0]),(ppoints[i][j][1])),radius, (255,0,0),-1 ) # Unet
            # out = cv2.addWeighted(img, alpha, lm_unet, 1 - alpha, 0)            
            # cv2.circle(img,((Predict_Points[i][j][0]),(Predict_Points[i][j][1])),radius,(0,0,255),-1) # RetinaNet
            cv2.circle(img,((GT_Points[i][j][0]),(GT_Points[i][j][1])),radius,(0,255,0),-1 ) # GT
        except:
            pass
#             print('error_fname_i_j',image_paths[i],i,j)

    for j in range(max_idx): # Unet numbering
        try:
            h = ppoints[i][j][1]
            w = ppoints[i][j][0]
            # font
            ((text_width, text_height), _) = cv2.getTextSize(str(j), cv2.FONT_HERSHEY_SIMPLEX, 1.1, 5)    

            cv2.putText(img, 
                text=str(j+1),
                # org=(w- int(0.5 * text_width), h - int(0.5 * text_height)),
                org=(w- int(0.7 * text_width), h - int(1.0 * text_height)),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=1.5, 
                color=(255,0,0),               
                thickness=2,
                lineType=cv2.LINE_AA,
            )
        except:
            pass
    
#     for j in range(max_idx): # Retina numbering
#         try:
#             h = Predict_Points[i][j][1]
#             w = Predict_Points[i][j][0]
#             # font
#             ((text_width, text_height), _) = cv2.getTextSize(str(j), cv2.FONT_HERSHEY_SIMPLEX, 1.1, 5)    

#             cv2.putText(img, 
#                 text=str(j+1),
#                 org=(w- int(0.5 * text_width), h - int(0.5 * text_height)),
#                 fontFace=cv2.FONT_HERSHEY_SIMPLEX,
#                 fontScale=1.1, 
#                 color=(0,0,255), #B                 
#                 thickness=2,
#                 lineType=cv2.LINE_AA,
#             )
#         except:
#             pass
        
    for j in range(max_idx): # GT numbering
        try:
            h = GT_Points[i][j][1]
            w = GT_Points[i][j][0]
            # font
            ((text_width, text_height), _) = cv2.getTextSize(str(j), cv2.FONT_HERSHEY_SIMPLEX, 1.1, 5)    

            cv2.putText(img, 
                text=str(j+1),
                org=(w- int(0.5 * text_width), h - int(0.5 * text_height)),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=1.1, 
                color=(0,255,0), #G
                thickness=2,
                lineType=cv2.LINE_AA,
            )
        except:
            pass
    
    plt.figure(figsize=(16, 16))
    plt.title('Retina : Red, U-Net : Blue, GT : Green')
    plt.axis('off')
    plt.imshow(img)
    plt.show()
    
    print(save_path + image_paths[i].split("/")[-1], img.shape)
    cv2.imwrite(save_path + image_paths[i].split("/")[-1], cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
    
np.save(save_path+'Unet_xy_coordinate.npy',ppoints)
np.save(save_path+'RetinaNet_xy_coordinate.npy',np.array(Predict_Points))

In [ ]:
# print(ppoints.shape)

# import pandas as pd
# lst = range(0,25)
# df = pd.DataFrame([ppoints[...,idx,0] for idx in lst])
# df = df.T
# df.to_csv('Unet_ROI'+str(patch_size)+'_x.csv')# please write save code 
# # df.to_csv('LERA_Retina_ROI'+str(patch_size)+'_x.csv')# please write save code 

# df = pd.DataFrame([ppoints[...,idx,1] for idx in lst])
# df = df.T
# df.to_csv('Unet_ROI'+str(patch_size)+'_y.csv')# please write save code 
# # df.to_csv('LERA_Retina_ROI'+str(patch_size)+'_y.csv')# please write save code 

# df

In [ ]:
# import pydicom
# files = natsorted(glob.glob('/workspace/flat_feet/dataset/Segmentation_210712/AMC_100_WBLR/original/*.dcm'))
# spacings = list()
# for idx in range(len(files)):
#     dcm = pydicom.dcmread(files[idx])
#     spacings.append(dcm.ImagerPixelSpacing)
# spacings = np.array(spacings)

In [ ]:
GT_Points = np.array(GT_Points)

In [ ]:
ppoints.shape,GT_Points.shape

In [ ]:
# GT_Points = np.load('../result/1024_KIM/Unet_xy_coordinate.npy')
ppoints = np.load('../result/1024_KIM/Unet_xy_coordinate.npy')

In [ ]:
name = 'UnetwoSCSEimport pandas as pd
_ROI1024'
lst = range(0,25)
df = pd.DataFrame([result_delta[...,idx,0] for idx in lst])
df = df.T
df.to_csv(name+str(patch_size)+'_x.csv',index=False) 

df = pd.DataFrame([result_delta[...,idx,1] for idx in lst])
df = df.T
df.to_csv(name+str(patch_size)+'_y.csv',index=False)

df = pd.DataFrame([result_distance[...,idx] for idx in lst])
df = df.T
df.to_csv(name+str(patch_size)+'_distance.csv',index=False)

# distance evaluation

In [2]:
def eval_distance(yhat,y):
    result_delta = np.zeros_like(y).astype(np.float32)
    result_distance = result_delta.copy()[...,0].astype(np.float32)
    cases = list()
    
    # 테스트셋의 경우 이미지 전치리하기를 x,y pixel spacing을 0.15로 맞췄음! 
    dx = 0.15
    dy = 0.15
    # dx = 1
    # dy = 1
    
    for idx_image in range(len(result_delta)):
        case = 'A' if y[idx_image][6][0] < y[idx_image][7][0] else 'P' # A for anteria, P for posteria   바닥의 점을 기준으로 anteria posteria 구분, (x = 0, y = 0) 은 좌측 상단 
        direction = 1 if case == 'A' else -1
        cases.append(case)
        for idx_lm in range(result_delta.shape[1]):
            delta_x = (- y[idx_image][idx_lm][0] + yhat[idx_image][idx_lm][0])*direction
            delta_y = - y[idx_image][idx_lm][1] + yhat[idx_image][idx_lm][1]
            result_delta[idx_image][idx_lm][0] = (delta_x*dx).astype(np.float32)
            result_delta[idx_image][idx_lm][1] = -(delta_y*dy).astype(np.float32)
            result_distance[idx_image][idx_lm] = np.sqrt((dx*delta_x)**2+(dy*delta_y)**2)
    
    return result_delta, result_distance, cases
    
result_delta, result_distance, cases = eval_distance(ppoints, GT_Points) # Unet 
# result_delta, result_distance, cases = eval_distance(Predict_Points, GT_Points) # RetinaNet

NameError: name 'ppoints' is not defined

In [ ]:
# tstat,pvalue = scipy.stats.ttest_rel(df1[f'{idx}'], df2[f'{idx}'])
# tstat,pvalue = scipy.stats.ttest_rel(np.abs(df1[f'{idx}']), np.abs(df2[f'{idx}']))

In [3]:
import scipy
import scipy.stats

def compute_pairedT(df1,df2,absolute=False):
    print('absolute',absolute)
    df1 = df1.round(4)
    df2 = df2.round(4)
    
    tstats = []
    pvalues = []

    for idx in range(0,3):
        if absolute:
            tstat,pvalue = scipy.stats.ttest_rel(np.abs(df1[f'{idx}']), np.abs(df2[f'{idx}']),axis=0)
        else:
            tstat,pvalue = scipy.stats.ttest_rel(df1[f'{idx}'], df2[f'{idx}'],axis=0)
        tstats.append(tstat)
        pvalues.append(pvalue)
    df1_stack = []
    df2_stack = []
    for idx in range(0,3):
        df1_stack.extend(df1[f'{idx}'])
        df2_stack.extend(df2[f'{idx}'])
    if absolute:
        tstat,pvalue = scipy.stats.ttest_rel(np.abs(df1_stack), np.abs(df2_stack),axis=0)
    else:
        tstat,pvalue = scipy.stats.ttest_rel(df1_stack, df2_stack,axis=0)
    tstats.append(tstat)
    pvalues.append(pvalue)    

    for idx in range(3,8):
        if absolute:
            tstat,pvalue = scipy.stats.ttest_rel(np.abs(df1[f'{idx}']), np.abs(df2[f'{idx}']),axis=0)
        else:
            tstat,pvalue = scipy.stats.ttest_rel(df1[f'{idx}'], df2[f'{idx}'],axis=0)
        tstats.append(tstat)
        pvalues.append(pvalue)
    df1_stack = []
    df2_stack = []
    for idx in range(3,8):
        df1_stack.extend(df1[f'{idx}'])
        df2_stack.extend(df2[f'{idx}'])
    if absolute:
        tstat,pvalue = scipy.stats.ttest_rel(np.abs(df1_stack), np.abs(df2_stack),axis=0)
    else:
        tstat,pvalue = scipy.stats.ttest_rel(df1_stack, df2_stack,axis=0)
    tstats.append(tstat)
    pvalues.append(pvalue)
    

    for idx in range(8,12):
        if absolute:
            tstat,pvalue = scipy.stats.ttest_rel(np.abs(df1[f'{idx}']), np.abs(df2[f'{idx}']),axis=0)
        else:
            tstat,pvalue = scipy.stats.ttest_rel(df1[f'{idx}'], df2[f'{idx}'],axis=0)
        tstats.append(tstat)
        pvalues.append(pvalue)
    for idx in range(20,25):
        if absolute:
            tstat,pvalue = scipy.stats.ttest_rel(np.abs(df1[f'{idx}']), np.abs(df2[f'{idx}']),axis=0)
        else:
            tstat,pvalue = scipy.stats.ttest_rel(df1[f'{idx}'], df2[f'{idx}'],axis=0)
        tstats.append(tstat)
        pvalues.append(pvalue)
    df1_stack = []
    df2_stack = []
    for idx in range(8,12):
        df1_stack.extend(df1[f'{idx}'])
        df2_stack.extend(df2[f'{idx}'])
    for idx in range(20,25):
        df1_stack.extend(df1[f'{idx}'])
        df2_stack.extend(df2[f'{idx}'])
    if absolute:
        tstat,pvalue = scipy.stats.ttest_rel(np.abs(df1_stack), np.abs(df2_stack),axis=0)
    else:
        tstat,pvalue = scipy.stats.ttest_rel(df1_stack,df2_stack,axis=0)
    
    # print('df1_stack')
    # for idx in range(len(df1_stack)):
    #     print(np.abs(df1_stack[idx]))
    # print('\n df2_stack')
    # for idx in range(len(df2_stack)):
    #     print(np.abs(df2_stack[idx]))
    tstats.append(tstat)
    pvalues.append(pvalue)
    

    for idx in range(12,20):
        if absolute:
            tstat,pvalue = scipy.stats.ttest_rel(np.abs(df1[f'{idx}']), np.abs(df2[f'{idx}']),axis=0)
        else:
            tstat,pvalue = scipy.stats.ttest_rel(df1[f'{idx}'], df2[f'{idx}'],axis=0)
        tstats.append(tstat)
        pvalues.append(pvalue)
    df1_stack = []
    df2_stack = []
    for idx in range(12,20):
        df1_stack.extend(df1[f'{idx}'])
        df2_stack.extend(df2[f'{idx}'])
    if absolute:
        tstat,pvalue = scipy.stats.ttest_rel(np.abs(df1_stack), np.abs(df2_stack),axis=0)
    else:
        tstat,pvalue = scipy.stats.ttest_rel(df1_stack,df2_stack,axis=0)
    tstats.append(tstat)
    pvalues.append(pvalue)
    
    df1_stack = []
    df2_stack = []
    for idx in range(0,25):
        df1_stack.extend(df1[f'{idx}'])
        df2_stack.extend(df2[f'{idx}'])
        
    if absolute:
        tstat,pvalue = scipy.stats.ttest_rel(np.abs(df1_stack), np.abs(df2_stack),axis=0)
    else:
        tstat,pvalue = scipy.stats.ttest_rel(df1_stack,df2_stack,axis=0)
        
    # print('df1_stack')
    # for idx in range(len(df1_stack)):
    #     print(np.abs(df1_stack[idx]))
    # print('\n df2_stack')
    # for idx in range(len(df2_stack)):
    #     print(np.abs(df2_stack[idx]))
    
    tstats.append(tstat)
    pvalues.append(pvalue)
    
    df = pd.DataFrame([tstats,pvalues],['tstats','pvalues'])
    df = df.T
    return df

In [14]:
coord = 'x'
coord = 'y'
coord = 'distance'
absolute = True

path1 = f'Unet_ROI512_{coord}.csv'
path1 = f'Unet_ROI1024_{coord}.csv'

paths2 = [f'Unet_ROI1024_{coord}.csv',
          f'UnetwoSCSE_ROI1024_{coord}.csv',
          f'Kim1024_{coord}.csv',
          f'FPAM_{coord}.csv',]

for path2 in paths2:
    df1 = pd.read_csv(path1)
    df2 = pd.read_csv(path2)
    
    print('\n')
    print(path1,len(df1),path2,len(df2))
    df = compute_pairedT(df1,df2,absolute=absolute)
    for idx in range(len(df['pvalues'])):
        print('{:.3f}'.format(df['pvalues'][idx]))



Unet_ROI1024_distance.csv 150 Unet_ROI1024_distance.csv 150
absolute True
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


Unet_ROI1024_distance.csv 150 UnetwoSCSE_ROI1024_distance.csv 150
absolute True
0.001
0.000
0.072
0.000
0.000
0.467
0.000
0.000
0.028
0.000
0.711
0.005
0.007
0.003
0.104
0.014
0.353
0.000
0.100
0.000
0.002
0.000
0.000
0.222
0.000
0.152
0.919
0.005
0.000
0.000


Unet_ROI1024_distance.csv 150 Kim1024_distance.csv 150
absolute True
0.096
0.851
0.540
0.804
0.236
0.386
0.004
0.078
0.797
0.655
0.001
0.007
0.776
0.000
0.000
0.147
0.000
0.000
0.009
0.000
0.063
0.357
0.001
0.094
0.000
0.226
0.000
0.248
0.000
0.000


Unet_ROI1024_distance.csv 150 FPAM_distance.csv 150
absolute True
0.002
0.000
0.459
0.443
0.203
0.911
0.572
0.000
0.840
0.037
0.604
0.023
0.025
0.198
0.000
0.099
0.740
0.027
0.033
0.018
0.429
0.221
0.787
0.428
0.062
0.002
0.124
0.599
0.096
0.625
